# Анализ рынка заведений общественного питания г.Москвы
**Цель проекта**: исследовать рынок заведений общественного питания для дальнейшего открытия заведения.

**Задачи проекта**:
- Исследовать существующие виды общественного питания, их распространненость;
- Изучить сетевые заведения города;
- Определить, какие районы наиболее выгодны для открытия заведения;
- Выявить, сколько посадочных мест является оптимальным.

Для реализации проекта будут использованы:  открытые данные о заведениях общественного питания в Москве и инструменты Python.

**Этапы исследования**:
1. [Подготовка данных](#preanalysis)
2. [Анализ данных](#analysis)
    - [Исследование видов объектов общественного питания](#type_object)
    - [Исследование сетевых и несетевых заведений](#chain)
    - [Изучение наиболее популярных улиц для заведений](#popular_street)
    - [Анализ наименее популярных районов для заведений](#unpopular_street)
    - [Изучение количества посадочных мест заведения на популярных улицах](#number_popular_street)
3. [Вывод и рекомендации](#conclusion)

## [Подготовка данных](#preanalysis)

На этапе подготовки данных импортируем все необходимые для реализации проекта библиотеки и откроем данные таблиц, чтобы оценить, какие изменения в данных нужно сделать.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
from plotly import graph_objects as go
import plotly.express as px
import requests

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
try:
    rest_data = pd.read_csv('/content/rest_data.csv')
except:
    rest_data = pd.read_csv('/datasets/rest_data.csv')
rest_data.info()
rest_data.head(10)

Явных пропусков в данных нет. Посмотрим детально корректность значений по столбцам. В названии заведений нужно будет поменять регистр и привести их к общему виду, а также в названии типа заведения и адресе. Помимо этого посмотрим уникальные значения столбцов, чтобы выявить неточности.

In [ ]:
rest_data['chain'].unique()


In [ ]:
rest_data['number'].unique()

In [ ]:
rest_data['object_type'].unique()

In [ ]:
rest_data['object_type'].value_counts()

In [ ]:
rest_data['object_type'] = rest_data['object_type'].str.replace('предприятие быстрого обслуживания', 'фаст-фуд')
rest_data['object_type'] = rest_data['object_type'].str.replace('магазин \(отдел кулинарии\)', 'кулинария')

In [ ]:
#приведем данные столбца с названиями заведений к нижнему регистру и проверим наличие дубликатов
rest_data['object_name'] = rest_data['object_name'].str.lower()
rest_data['object_name'] = rest_data['object_name'].str.replace('ё','е')

#избавимся от типа заведения в названии
for type_rest in ['кафе', 'бар', 'кальянная', 'ресторан', 'буфет', 'столовая', 'хинкальная']:
    rest_data['object_name'] = rest_data['object_name'].str.replace(type_rest,'', regex=True).str.strip()

rest_data['object_name'] = rest_data['object_name'].str.replace("«",'', regex=True).str.strip()
rest_data['object_name'] = rest_data['object_name'].str.replace("»",'', regex=True).str.strip()
rest_data['object_name'] = rest_data['object_name'].str.replace("-",'', regex=True).str.strip()

rest_data['object_name'].unique()

In [ ]:
rest_data['address'] = rest_data['address'].str.replace('ё','е')
rest_data['address'] = rest_data['address'].str.lower()

In [ ]:
rest_data.duplicated().sum()

В ходе подготовки данных к анализу данные были приведены к общему виду для повышения точности исследования.

## [Анализ данных](#analysis)
**[Исследование видов объектов общественного питания](#type_objec)**

На этом этапе проведем анализ заведений, отталкиваясь от того, к какому виду они относятся. В начале проанализируем соотношение видов объектов общественного питания по количеству заведений.

In [ ]:
object_type_count = pd.DataFrame(rest_data.groupby('object_type')['id'].count()).reset_index().sort_values(by='id', ascending=False)
object_type_count

In [ ]:
plt.figure(figsize=(16,8))
sns.set_palette('colorblind') 
sns.set_style("whitegrid")
ax = sns.barplot(data=object_type_count, x="id", y="object_type", estimator=np.mean)
ax.set_title("Соотношение видов объектов общественного питания по количеству")
ax.set_ylabel("Вид заведения")
ax.set_xlabel("Посетители, чел.")
plt.show()
plt.close()

Больше всего в Москве кафе, столовых, ресторанов и фаст-фуда. Кафе лидируют, скорее всего, за счет своей доступности: не очень высокий средний чек, комфортный уровень обслуживания. Кулинарии и закусочных почти нет. Возможно это связано с интрепретацией данных, т.к. вид заведения может быть определен не совсем кореектно. Псомотрим, как меняется среднее количество посадочных мест в зависимости от типа заведения.

In [ ]:
rest_data['number'].describe()

График среднего значения будем строить по медианным данным, т.к. могут быть искажения за счет большого разброса между минимальным и максимальным значениями.

In [ ]:
object_type_number = pd.DataFrame(rest_data.groupby('object_type')['number'].median()).reset_index()
object_type_number

In [ ]:
plt.figure(figsize=(16,8))
sns.set_palette('colorblind') 
sns.set_style("whitegrid")
ax = sns.boxplot(data=rest_data, x="number", y="object_type")
ax.set_title("Среднее количество посадочных мест в зависимости от вида заведения")
ax.set_xlim(0,600)
ax.set_ylabel("Вид заведения")
ax.set_xlabel("Посетители, чел.")
plt.show()
plt.close()

В среднем самое большое количество посадочных мест в столовых и ресторанах. Столовые рассчитаны на большое количество посетителей, часто они находятся при каких-то организациях, откуда много сотрудников идут на обед именно в столовую. Рестораны славятся не только едой, но и интерьером, поэтому их часто тоже делают в больших помещениях + немаловажный фактор что часто в ресторанах и столовых проводят мероприятия. Меньше всего вероятность найти место - в заведениях фаст-фуда и кафетерии. Данных по местам в кулинарии и закусочной нет.

**[Исследование сетевых и несетевых заведений](#chain)**

Заведения общественного питания делятся на сетевых и несетевые. Посмотрим, как распределилось соотношение сетевых и несетевых заведений по количеству заведений в городе. 

In [ ]:
chain_type_count = pd.DataFrame(rest_data.groupby('chain')['id'].count()).reset_index().sort_values(by='id', ascending=False)
chain_type_count

In [ ]:
name_rest = 'Несетевые', 'Сетевые'
values = chain_type_count['id']

fig = go.Figure(data=[go.Pie(labels=name_rest, values=values, pull=[0, 0.2])])
fig.update_traces(textposition='inside')
fig.update(layout_title_text='Соотношение сетевых и несетевых заведений общественного питания', 
           layout_showlegend=True)
fig.show()

Несетевые заведения составляют почти 81% всех заведений города. Сетевые, скорее всего, сложно поддерживать, т.к. зачастую одно сетевое заведение приносит много прибыли, а второе - работает в убыток, за счет чего компания скорее работает в ноль. Посмотрим, где больше посадочных мест: в сетевых или несетвых заведениях и проанализируем сетевые заведения по количеству мест в них.

In [ ]:
chain_type_number = pd.DataFrame(rest_data.groupby('chain')['number'].mean()).reset_index().sort_values(by='number', ascending=False)
chain_type_number

In [ ]:
name_rest = 'Несетевые', 'Сетевые'
values = chain_type_number['number']

fig = go.Figure(data=[go.Pie(labels=name_rest, values=values)])
fig.update_traces(textposition='inside')
fig.update(layout_title_text='Среднее количество посадочных мест', 
           layout_showlegend=True)
fig.show()

In [ ]:
yes = 'да'
is_chain = rest_data.query('chain == @yes')

is_chain['object_type'].unique().tolist()

In [ ]:
chain_object = is_chain.groupby('object_type').agg({'number' : 'median', 'id' : 'count'}).sort_values(by='number', ascending=False).reset_index()
chain_object

In [ ]:
plt.figure(figsize=(16,8))
sns.set_palette('Set3') 
ax = sns.boxplot(x=chain_object['number'])
ax.set_xlabel("Посадочные места, шт.")
plt.show()
plt.close()

In [ ]:
chain_object['number'].describe()

In [ ]:
plt.figure(figsize=(16,8))
sns.set_palette('colorblind') 
ax = sns.boxplot(x=chain_object['id'])
ax.set_xlabel("Посетители, чел.")
plt.show()
plt.close()

In [ ]:
chain_object['id'].describe()

Для сетевых заведений характерно много заведений с небольшим числом посадочных мест и, соответсвенно, небольшим числом посетителей. Но есть заведения, сильно отклоняющиеся от данной тенденции.

In [ ]:
v

Сетевыми чаще всего являются кафе, фаст-фуд и рестораны.

In [ ]:
(rest_data
         .pivot_table(index='object_type', columns = 'chain',values='id', aggfunc='mean')
         .plot(kind='bar', grid=True, figsize=(16,8), rot=45, title='Сравнение сетевых и несетвых заведений по среднему количеству посетителей на вид заведения', color=['cadetblue','peru'])
);

Большая часть заведений общественного питания в Москве - несетевые. Среди несетвых заведений в среднем больше всего посетителей приходится на: закусочные, кафетерии, кулинарии и фаст-фуды. Среди сетевых -буфеты, кафетерии и кулинарии.  

**[Изучение наиболее популярных улиц для заведений](#popular_street)**

На основе имеющихся данных об адресе заведения, выделим 10 наиболее популярных месторасположений, а далее разделим их по районам.

In [ ]:
list = (['город московский', 'поселение мосрентген', 'поселение московский', 'поселение михайлово-ярцевское', 'поселение марушкинское', 'город троицк', 'поселение сосенское', 'город зеленоград'])
def get_street(row):
      if row['address'].split(",")[0].strip() == 'город москва' and row['address'].split(",")[1].strip() not in list:
            return row['address'].split(",")[1].strip()
      elif row['address'].split(",")[1].strip() in list:
            return row['address'].split(",")[2].strip()
      else:
            return row['address'].split(",")[0].strip()
rest_data['street'] = rest_data.apply(get_street, axis=1)
rest_data

In [ ]:
street_count = rest_data.groupby('street').agg({'id': 'count', 'number' : 'median'}).reset_index().sort_values(by='id', ascending=False)
street_count_head = street_count.head(10)
street_count_head

In [ ]:
plt.figure(figsize=(16,8))
sns.set_palette('muted') 
sns.set_style("whitegrid")
ax = sns.barplot(data=street_count_head, y="street", x="id")
ax.set_title("Топ-10 улиц по количеству объектов общественного питания")
ax.set_ylabel("")
ax.set_xlabel("Заведения общественного питания, шт.")
plt.show()
plt.close()

Распределение наиболее густозаселенных заведениями улиц по районам и округам:
* Проспект Мира: СВАО(а именно: Мещанский, Марьина Роща, Алексеевский, Останкинский, Свиблово, Ростокино)
* Профсоюзная улица: ЮЗАО (а именно: Академический, Коньково, Обручевский, Тёплый Стан, Черёмушки, Ясенево)
* Ленинградский проспект: САО (а именно: Беговой, Аэропорт, Сокол, Хорошёвский)
* Пресненская набережная: ЦАО (а именно: Пресненский)
* Варшавское шоссе: ЮЗАО (а именно: Донской, Нагатино-Садовники, Нагорный)
* Ленинский проспект: ЮЗАО (а именно: Гагаринский и Донской)
* Проспект Вернадского: ЗАО (а именно: Проспект Вернадского)
* Кутузовский проспект: ЗАО (а именно: Дорогомилово, Фили-Давыдково)
* Каширское шоссе: ЮАО (а именно: Нагатино-Садовники и Нагорный)
* Кировоградская улица: ЮАО (а именно: Чертаново Северное, Чертаново Центральное и Чертаново Южное)

Наиболее популярные округа для заведений общепита: ЮЗАО, ЗАО, ЮАО, а именно юг оранжевой и серой ветки г. Москвы, не прилегающий к центру города, находятся в 3-4 станциях от кольцевой. То есть чаще открываются заведения в спальных районах, скорее всего в расчете на то, что люди скорее зайдут в ближайшее заведение, чем поедут куда-то далеко.

**[Анализ наименее популярных районов для заведений](#unpopular_street)**

Посмотрим, на каких улицах располагается только одно заведение и изучим, в каких районах располагаются такие заведения.

In [ ]:
unpopular_street = street_count.query('id == "1"')
print('Всего улиц с 1 объектом общественного питания: {}, что составляет {:.1%} от всего кол-ва улиц с объектами общественного питания'\
      .format(len(unpopular_street), (len(unpopular_street)/len(street_count))))

In [ ]:
from io import BytesIO
import requests
spreadsheet_id = '1gKh1MjZk6oa_b7qggRm9wGai_nLfbDIhDMhwWscqCao'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
district = pd.read_csv(BytesIO(r.content))
district.head(10)

In [ ]:
district['streetname'] = district['streetname'].str.lower()
district

In [ ]:
unpopular_merged = unpopular_street.merge(district[['streetname', 'okrug', 'area']], left_on='street', right_on='streetname')
display(unpopular_merged.groupby('area')['id'].count().reset_index().sort_values(by='id').head(10))

In [ ]:
display(unpopular_merged.groupby('area')['id'].count().reset_index().sort_values(by='id').tail(10))

Всего улиц с 1 объектом общественного питания: 652, что составляет 32.1% от всего кол-ва улиц с объектами общественного питания.

Больше всего улиц с одним объектом общественного питания в центре города: Таганский, Хамовники, Басманный, Пресненский, Тверской. Возможно одна из причин такого распределения - много маленьких улиц в центре. По одному объекту общесвтенного питания было определено в спальных районах, среди них встречаются районы, которые вошли в список наиболее популярных районов, что возможно связано с привязкой улицы к району, где могут быть некоторые неточности.

**[Изучение количества посадочных мест заведения на популярных улицах](#number_popular_street)**

In [ ]:
top10_streets_list = street_count_head['street'].to_list()
top10_streets_data = rest_data.query('street in @top10_streets_list')

In [ ]:
top10_streets_data['number'].median()
rest_data['number'].median()
plt.figure(figsize=(16,8))
sns.set_palette('muted') 
sns.set_style("whitegrid")
ax = sns.boxplot(data=top10_streets_data, x='number', y='street')
ax.set_xlim(-10,700)
plt.title('Распределение количества посадочных мест для топ-10 улиц по количеству объектов')
plt.ylabel('')
plt.xlabel('Посадочные места, шт.')
plt.show()

В среднем на самых густонаполненных заведениями улицах - 20-50 посадочных мест, а выбросы для каждой улицы начинаются после 150 посадочных мест.

## [Вывод и рекомендации](#conclusion)

* Больше всего в Москве кафе, столовых, ресторанов и фаст-фуда;
* Кулинарии и закусочные наименее популярные виды общепита в Москве;
* В среднем самое большое количество посадочных мест в столовых и ресторанах;
* Зведения фаст-фуда и кафетерии обычно не располагают большим количеством посадочных мест;
* Несетевые заведения составляют почти 81% всех заведений города;
* Для сетевых заведений характерно много заведений с небольшим количеством посадочных мест;
* Сетевыми чаще всего являются кафе, фаст-фуд и рестораны;
* Наиболее популярные районы для заведений общепита:
      - Московский международный деловой центр Москва-Сити;
      - район Чертаново Центральное;
      - район Тропарёво-Никулино;
      - Мещанский район;
      - район Свиблово;
      - район Аэропорт;
      - район Дорогомилово;
      - район Нагатино-Садовники;
      - район Фили-Давыдково;
      - Алексеевский район.
* Наименее популярные районы с большим количеством улиц с одним заведением находятся в основном в центре:
      - Таганский район - 27 улиц с одним объектом;
      - Тверской район - 23 улицы с одним объектом;
      - Басманный район - 23 улицы с одним объектом;
      - район Марьина Роща - 19 улиц с одним объектом;
      - Пресненский район - 17 улиц с одним объектом;
      - район Богородское - 13 улиц с одним объектом;
      - Даниловский район - 12 улиц с один объектом;
      - Мещанский район - 12 улиц с одним объектом;
      - район Замоскворечье - 12 улиц с одним объектом;
      - район Соколиная гора - 11 улиц с одним объектом.
* В среднем в популярных райнах среди заведений находятся заведения с 20-50 посадочными местами.

**На основе полученной информации можно говорить о том, что:**
* Выбирая тип заведения, лучше остановиться на кафе или ресторане. Да, их много, но они популярны.
* Распространение несетвых заведений общепита выше, чем сетевых, так что скорее всего сетевые заведения менее выгодны и за счет этого они чаще закрываются в том или ином месте.
* Заведение лучше открывать не в центре города, а в относительно спальных районах, если точнее, то: район Чертаново, Свиблово, Аэропорт, Деловой центр, Тропарёво-Никулино. В этих районах высокая концентрация заведений, за счет чего там выше проходимсть и при правильно построенной концепции и маркетинге - можно привлечь большое количество посетителей. 

Ссылка на презентацию: https://disk.yandex.ru/i/Yd4jD6_hjdWOPA
